In [3]:
import os
from nltk.tag import StanfordNERTagger as NERTagger
from nltk.tag import StanfordPOSTagger as POSTagger
ner_jar = 'StanfordNLP/jars/stanford-ner.jar'
ner_model = 'StanfordNLP/models/english.all.3class.distsim.crf.ser.gz'
pos_jar = 'StanfordNLP/jars/stanford-postagger.jar'
pos_model = 'StanfordNLP/models/english-bidirectional-distsim.tagger'
ner_tagger = NERTagger(ner_model, ner_jar)
pos_tagger = POSTagger(pos_model, pos_jar)

In [5]:
import pickle
dic_freq = []
companies = ['AAPL', 'GOOG', 'MSFT', 'CSCO', 'FB']
for comp in companies:
    with open('pickles/{}/{}_2017.pickle'.format(comp, comp), 'rb') as f:
        dic_freq.append(pickle.load(f))
word_list = []
for dic in dic_freq:
    word_list.append([i[0] for i in dic])
for wl in word_list:
    print(wl[:20])

['the', 'and', 'Company', 'for', 'any', 'that', 'with', 'are', 'its', 'may', 'other', 'The Company', 'such', 'not', 'under', 'shall', 'Plan', 'which', 'sales', 'net']
['and', 'the', 'our', 'for', 'are', 'that', 'from', 'revenues', 'other', 'with', 'income', 'have', 'December', 'not', 'may', 'tax', 'net', 'which', 'financial', 'stock']
['and', 'the', 'our', 'for', 'revenue', 'are', 'that', 'with', 'from', 'billion', 'services', 'income', 'other', 'products', 'year', 'may', 'million', 'business', 'fiscal', 'due']
['the', 'and', 'any', 'Company', 'for', 'that', 'with', 'you', 'our', 'shall', 'are', 'this', 'your', 'other', 'may', 'from', 'such', 'not', 'Plan', 'Shares']
['and', 'the', 'our', 'for', 'that', 'are', 'may', 'other', 'with', 'tax', 'from', 'products', 'not', 'have', 'financial', 'revenue', 'which', 'such', 'stock', 'income']


In [6]:
org_list = []
org_set = set()
loc_list = []
loc_set = set()
per_list = []
per_set = set()
spe_list = []
spe_set = []
for wl in word_list:
    wl_compound = list(filter(lambda word: ' ' in word, wl))
    spe_list.append(wl_compound)
    spe_set += wl_compound
    #wl = list(filter(lambda word: ' ' not in word, wl))
    dic_ner = dict(ner_tagger.tag(wl))
    org_list.append([])
    loc_list.append([])
    per_list.append([])
    for word, ner in dic_ner.items():
        if len(word) < 3:
            continue
        if ner == 'ORGANIZATION':
            org_list[-1].append(word)
            org_set.add(word)
        if ner == 'LOCATION':
            loc_list[-1].append(word)
            loc_set.add(word)
        if ner == 'PERSON':
            per_list[-1].append(word)
            per_set.add(word)
spe_set = set(spe_set)
print(list(org_set)[:20])
print(list(loc_set)[:20])
print(list(per_set)[:20])
print(list(spe_set)[:20])

['Morocco', 'Federal', 'Guatemala', 'Department', 'Analytics', 'Europe', 'Singapore', 'Charles', 'European', 'Jones', 'SQL', 'Ireland', 'SRI', 'Unified', 'Nacional', 'hardware', 'Conferencing', 'Sandberg', 'Customer', 'Nexus']
['Taiwan', 'Arabia', 'Pakistan', 'Europe', 'Russian', 'Denmark', 'Fargo', 'APJC', 'Dakota', 'Pacific', 'singapore', 'America', 'While', 'Malaysia', 'japan', 'Limitada', 'Thailand', 'Road', 'Florida', 'Nigeria']
['KONDO', 'Kathleen', 'Wagner', 'Frank', 'Signature', 'Date', 'Charles', 'James', 'Eric', 'Koum', 'Deloitte', 'Treadway', 'SUSAN', 'Chris', 'Jean-Philippe', 'CHRIS', 'Levinson', 'JUNG', 'andreessen', 'Daniel']
['Contents Alphabet Inc Alphabet Inc NOTES TO CONSOLIDATED FINANCIAL STATEMENTS Note', 'EQUITY GRANT INITIAL ANNUAL RETAINER The', 'Photos Safari®', 'James G Campbell', 'Chief Legal Officer Mr Nadella', 'Receivables Receivables Contracts Total Gross', 'Advanced Malware Protection AMP', 'CHARLES W SCHARF Director Charles W Scharf', 'Cumulative Total R

In [7]:
adj_list = []
adj_set = set()
for wl in word_list:
    dic_pos = dict(pos_tagger.tag(wl[:200]))
    adj_list.append([])
    for word, pos in dic_pos.items():
        if len(word) < 3:
            continue
        if pos == 'JJ':
            adj_list[-1].append(word)
            adj_set.add(word)
print(list(adj_set)[:20])

['third', 'significant', 'Tax-Related', 'affected', 'deferred', 'third-party', 'pursuant', 'common', 'subject', 'technical', 'additional', 'certain', 'registrant', 'Registrant', 'mobile', 'issued', 'consolidated', 'due', 'future', 'effective']


In [10]:
import numpy as np
import pandas as pd
org_matrix = np.array([[(dict(dic)[word] if word in dict(dic) else 0) for word in org_set] for dic in dic_freq])
org_matrix = pd.DataFrame(org_matrix, columns = org_set, index = companies)
org_matrix

,Morocco,Federal,Guatemala,Department,Analytics,Europe,Singapore,Charles,European,Jones,...,Estonia,Maxtor,Manufacturing-Related,Google,TOUCHE,Premium,Properties,LLP,Holding,Foundation
AAPL,0,1,0,0,0,11,0,0,6,0,...,0,0,0,0,0,0,0,0,0,0
GOOG,0,2,0,4,0,7,0,0,3,0,...,0,0,0,123,0,0,0,3,1,0
MSFT,0,1,0,0,0,4,5,2,3,0,...,0,0,0,5,2,1,0,2,0,0
CSCO,2,5,2,0,1,10,3,10,0,0,...,2,1,0,0,0,0,0,1,1,0
FB,0,2,0,0,0,16,0,0,2,0,...,0,0,0,2,0,0,0,0,0,1


In [11]:
per_matrix = np.array([[(dict(dic)[word] if word in dict(dic) else 0) for word in per_set] for dic in dic_freq])
per_matrix = pd.DataFrame(per_matrix, columns = per_set, index = companies)
per_matrix

,KONDO,Kathleen,Wagner,Frank,Signature,Date,Charles,James,Eric,Koum,...,Kling,Timothy,Arthur,Susan,Ronald,JOHN,Brod,Erskine,Schiller,CHARLES
AAPL,1,0,0,0,0,11,0,1,0,0,...,0,9,1,1,1,0,0,0,0,0
GOOG,0,0,0,0,0,4,0,1,7,0,...,0,0,0,0,0,1,0,0,0,0
MSFT,0,1,0,3,0,12,2,0,0,0,...,0,0,0,0,0,2,0,0,0,2
CSCO,0,0,0,0,2,26,10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FB,0,0,0,0,1,5,0,0,0,0,...,0,0,0,4,0,0,0,2,0,0


In [12]:
loc_matrix = np.array([[(dict(dic)[word] if word in dict(dic) else 0) for word in loc_set] for dic in dic_freq])
loc_matrix = pd.DataFrame(loc_matrix, columns = loc_set, index = companies)
loc_matrix

,Taiwan,Arabia,Pakistan,Europe,Russian,Denmark,Fargo,APJC,Dakota,Pacific,...,Turkey,Korea,San,Russia,Indonesia,Beijing,Canada,York,Kong,United
AAPL,4,0,0,11,0,0,0,0,0,0,...,0,0,4,0,0,0,0,1,0,0
GOOG,0,0,0,7,2,0,0,0,0,0,...,0,0,2,0,0,0,2,2,0,0
MSFT,0,0,0,4,0,0,1,0,1,0,...,0,1,0,0,0,1,3,23,0,6
CSCO,2,4,2,10,4,9,0,16,0,4,...,2,6,7,2,4,4,15,4,6,4
FB,0,0,0,16,1,0,0,0,0,0,...,3,1,2,1,6,0,5,2,0,2


In [25]:
unimportant = ['other', 'such']
for word in unimportant:
    if word in adj_set:
        adj_set.remove(word)
adj_matrix = np.array([[(dict(dic)[word] if word in dict(dic) else 0) for word in adj_set] for dic in dic_freq])
adj_df = pd.DataFrame(adj_matrix, columns = adj_set, index = companies)
adj_df

,third,significant,Tax-Related,affected,deferred,third-party,pursuant,common,subject,technical,...,10-K,based,included,cloud-based,otherwise,available,net,global,content,share-based
AAPL,35,54,4,33,42,53,86,55,175,7,...,126,92,41,0,61,58,211,31,47,14
GOOG,8,72,0,22,33,9,23,58,56,13,...,42,73,53,1,18,26,175,10,59,0
MSFT,13,74,0,15,29,14,7,30,38,7,...,37,72,83,34,5,37,93,32,25,0
CSCO,112,64,5,46,105,24,335,106,329,38,...,19,202,85,7,198,49,143,23,3,36
FB,36,75,0,38,33,12,18,135,71,31,...,39,66,36,0,26,27,97,8,54,45


In [27]:
adj_comatrix = np.matmul(np.transpose(adj_matrix), adj_matrix)
max_co = []
for i in range(adj_comatrix.shape[0]):
    for j in range(adj_comatrix.shape[1]):
        max_co.append((adj_comatrix[i][j], i, j))

adj_comatrix = pd.DataFrame(adj_comatrix, columns = adj_set, index = adj_set)

for i in sorted(max_co)[:-21:-1]:
    print(i[0], adj_comatrix.index[i[1]], adj_comatrix.columns[i[2]])
adj_comatrix

158179 financial financial
150175 fiscal fiscal
148487 subject subject
133705 applicable subject
133705 subject applicable
132949 applicable applicable
129897 applicable fiscal
129897 fiscal applicable
129660 fiscal subject
129660 subject fiscal
128882 financial subject
128882 subject financial
128097 subject pursuant
128097 pursuant subject
127505 net financial
127505 financial net
126505 applicable pursuant
126505 pursuant applicable
123286 fiscal pursuant
123286 pursuant fiscal


,third,significant,Tax-Related,affected,deferred,third-party,pursuant,common,subject,technical,...,10-K,based,included,cloud-based,otherwise,available,net,global,content,share-based
third,15298,13296,700,8046,15059,5229,41453,19511,46471,5812,...,8759,29740,13754,1234,25456,9179,29502,4445,4722,6142
significant,13296,23297,536,10270,15985,6982,29608,26275,42675,6589,...,16707,33430,20312,3036,19582,12903,47303,6834,12878,6435
Tax-Related,700,536,41,362,693,332,2019,750,2345,218,...,599,1378,589,35,1234,477,1559,239,203,236
affected,8046,10270,362,5358,8631,3717,19543,13547,25409,3548,...,7993,17522,9042,854,12580,6321,22472,3085,5414,3828
deferred,15059,15985,693,8631,15808,5845,40343,20679,47188,5939,...,11033,31749,15991,1754,24949,10403,35550,5239,6743,5853
third-party,5229,6982,332,3717,5845,3806,13119,8021,19059,1870,...,8498,12181,6284,653,8529,5326,18656,2829,4092,2146
pursuant,41453,29608,2019,19543,40343,13119,120523,44214,128097,14238,...,19128,78953,34449,2606,72493,22746,72473,10969,7551,14074
common,19511,26275,750,13547,20679,8021,44214,36750,58472,9562,...,17755,41776,21689,1820,29047,14647,52798,6763,14365,10661
subject,46471,42675,2345,25409,47188,19059,128097,58472,148487,16922,...,34828,94068,43818,3651,78861,31050,104193,15336,17300,17489
technical,5812,6589,218,3548,5939,1870,14238,9562,16922,2672,...,3618,11819,5903,517,9026,3702,12844,1693,3059,2861


In [31]:
loc_adj_comatrix = np.matmul(np.transpose(loc_matrix), adj_matrix)
max_co = []
for i in range(loc_adj_comatrix.shape[0]):
    for j in range(loc_adj_comatrix.shape[1]):
        max_co.append((loc_adj_comatrix[i][j], i, j))

loc_adj_comatrix = pd.DataFrame(loc_adj_comatrix, columns = adj_set, index = loc_set)

for i in sorted(max_co)[:-21:-1]:
    print(i[0], loc_adj_comatrix.index[i[1]], loc_adj_comatrix.columns[i[2]])
loc_adj_comatrix

39389 California applicable
39286 California subject
38601 California fiscal
37529 California pursuant
30354 California related
28394 California financial
24304 California based
22513 California otherwise
21863 California necessary
21691 California certain
21453 California net
20744 California issued
20723 California outstanding
19831 California future
18846 California additional
18162 California fair
17437 California due
17133 California foreign
16948 State applicable
16793 State subject


,third,significant,Tax-Related,affected,deferred,third-party,pursuant,common,subject,technical,...,10-K,based,included,cloud-based,otherwise,available,net,global,content,share-based
Taiwan,364,344,26,224,378,260,1014,432,1358,104,...,542,772,334,14,640,330,1130,170,194,128
Arabia,448,256,20,184,420,96,1340,424,1316,152,...,76,808,340,28,792,196,572,92,12,144
Pakistan,224,128,10,92,210,48,670,212,658,76,...,38,404,170,14,396,98,286,46,6,72
Europe,2189,3234,94,1645,2387,1134,4773,4351,6895,1072,...,2642,4887,2580,213,3213,1890,6900,897,1924,1234
Russian,500,475,20,266,519,126,1404,675,1499,209,...,199,1020,482,30,854,275,1019,120,184,189
Denmark,1008,576,45,414,945,216,3015,954,2961,342,...,171,1818,765,63,1782,441,1287,207,27,324
Fargo,13,74,0,15,29,14,7,30,38,7,...,37,72,83,34,5,37,93,32,25,0
APJC,1792,1024,80,736,1680,384,5360,1696,5264,608,...,304,3232,1360,112,3168,784,2288,368,48,576
Dakota,13,74,0,15,29,14,7,30,38,7,...,37,72,83,34,5,37,93,32,25,0
Pacific,448,256,20,184,420,96,1340,424,1316,152,...,76,808,340,28,792,196,572,92,12,144
